In [104]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_json("lessons_pd.json")
df.head(5)

,date,start,end,subject,class,teacher,classroom
0,28/11,570,650,Информатика,MIX-2-22,Апел Сарыбаева,LAB3
1,28/11,700,780,Математика,MIX-2-22,Ольга Беляева,C9
2,28/11,790,870,Основы креативного и критического мышления,MIX-2-22,Анастасия Шабалина,C9
3,28/11,880,960,Физика,MIX-2-22,Аманбек Чечейбаев,C9
4,28/11,970,1050,Дополнительный курс по математике,SEST-1-22,Жетиген Бакыт у.,C11


In [119]:
import datetime
all_cabs = set(df["classroom"].unique())
cabs = {cab for cab in all_cabs if not cab.isdigit() and cab[0] != "G"}
def time(hours, minutes):
    return hours*60+minutes
def to_time(entry):
    return entry["start"] =
def then(df, date, time):
    return df[(df["date"] == date) & (df["start"]<=time) & (df["end"]>=time)]
def now(df):
    today = datetime.datetime.now()
    return then(df, f"{today.day}/{today.month}", time(today.hour, today.minute))
def where_will(df, teacher, date):
    return df[df["teacher"] == teacher][df["date"] == date]
def where(df, teacher):
    today = datetime.datetime.now()
    return where_will(df, teacher, f"{today.day}/{today.month}")

In [125]:
cabs - set(now(df)["classroom"])

{'C8', 'CH'}